# 🐩 Poodle Palette – Genotype → Phenotype Tutor
Welcome! In this notebook you’ll learn how two key genes (B and E) create the classic Poodle coat colours—and you’ll see the result for *any* genotype you enter.

In [1]:
# 📦 Setup
# If running locally, make sure pandas and matplotlib are installed.
# In Google Colab these come pre‑installed.
import pandas as pd
import matplotlib.pyplot as plt

# Display plots inline
%matplotlib inline


## 🎓 2. Genetic Background
Poodle colours hinge mainly on two loci:

| Locus | Dominant / recessive | What it controls |
|-------|----------------------|------------------|
| **B (TYRP1)** | **B** = black pigment<br>**b** = brown pigment | `bb` turns all black areas brown (“liver”). |
| **E (MC1R)** | **E** = allows black/brown<br>**e** = blocks black → red/cream | `ee` dogs show only red/cream pigment. |

*We’ll ignore rarer loci (Dilute, Spotting) to keep the demo bite-sized.*

In [5]:
## 3. Imports + Helper Dictionaries


In [7]:
## 4. Function `predict_colour()`

In [8]:
## 5. Bulk Conversion of Your CSV

In [9]:
## 6. Interactive Playground

In [ ]:
## 7. Visual Summary

In [10]:
## 8. Further Reading